In [ ]:
import fitz

def get_table_bounding_boxes(pdf_path):
    doc = fitz.open(pdf_path)
    table_bounding_boxes = []
    text = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)

        blocks = page.get_text("dict")["blocks"]

        for block in blocks:
            # table_bounding_boxes.append(block['bbox'])
            if block['type'] == 0:  # 0 indicates a block of text
                for line in block['lines']:
                    # line_bbox = line['bbox']  # Bounding box for the line [x0, y0, x1, y1]
                    # table_bounding_boxes.append(line_bbox)
                    for span in line['spans']:
                        text.append(span['text'])
                        table_bounding_boxes.append(span['bbox'])

    return table_bounding_boxes, text

# Example usage
pdf_path = ""
table_bboxes, text_data = get_table_bounding_boxes(pdf_path)

In [4]:
out = open("output.txt", "w") # create a text output
for text in text_data: # iterate the document pages
    out.write(text + '\n') # write text of page
out.close()

In [8]:
doc = fitz.open(pdf_path)
page = doc[0]
for block in table_bboxes:
    rect = fitz.Rect(block[:4])  
    highlight = page.add_rect_annot(rect)
    highlight.set_colors(stroke=(1, 0, 0))  # Red border
    highlight.update()

doc.save("highlighted.pdf")
doc.close()


In [ ]:
import fitz  # PyMuPDF

def highlight_lines_in_pdf(pdf_path, output_path):
    # Open the PDF
    doc = fitz.open(pdf_path)

    # Loop through each page in the PDF
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)

        # Extract the drawing items (lines, curves, polygons, etc.)
        drawings = page.get_drawings()

        print(len(drawings))

        # Iterate through the drawing items to highlight lines
        count = 0
        for drawing in drawings:
            if drawing['items'][0][0] == 'l':
                point1, point2 = drawing['items'][0][1], drawing['items'][0][2]
                # print(item)
                # Check if the item is a line (lines are represented by 2 points: (x0, y0) to (x1, y1))
                #     # Extract the coordinates of the line
                x0, y0, x1, y1 = point1.x, point1.y, point2.x, point2.y
                    
                #     # Create a rectangle around the line (to highlight it)
                rect = fitz.Rect(x0, y0, x1, y1)

                #     # Highlight the line with a thicker line or rectangle (for visibility)
                highlight_color = (1, 1, 0)  # Yellow color
                page.draw_line((x0, y0), (x1, y1), color=highlight_color, width=2)  # Draw a thicker line for highlighting
        # Optionally, save the modified page or the full document
        doc.save(output_path)

# Example usage
pdf_path = ""
output_path = ""
highlight_lines_in_pdf(pdf_path, output_path)


16781


KeyboardInterrupt: 

In [24]:
from PIL import Image

image = Image.open("./public/output_ocr_file.png")
w, h = image.size

sections = 3
pieces = w // sections

for i in range(sections):
    image.crop((pieces * i, 0, pieces * (i + 1), h - 500)).save("./cropped-image.png")
    if i == 3:
        break